county weekno noofInvoice Totalqauntity and invoicevalue
first two are grouping column and remaining are aggregate columns 
however you dont have weekno - but you have invoicedate that you transform to weekno we have builtit function for it
NUofinvoice - countdistinct
Sum of qautity is totalqauntity
invoice value is qauntity into unitprice

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql import functions as f

spark = SparkSession.builder \
            .master("local[3]") \
            .appName("Misc Transformations") \
            .getOrCreate()

In [2]:
invoice_df = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("data/invoices.csv")

In [3]:
# lets define three aggregation as varibale
NumInvoices = f.countDistinct("InvoiceNo").alias("NumInvoices")
TotalQuantity = f.sum("Quantity").alias("TotalQuantity")
InvoiceValue = f.expr("round(sum(Quantity * UnitPrice),2) as InvoiceValue")

Covert the string date into date field to get date in format to calculate weekno
exSummary_df = invoice_df \
        .withColumn("InvoiceDate", f.to_date(f.col("InvoiceDate"), "dd-MM-yyyy H.mm")) \ # Invoice date is in proper date type
        #sample data littel big for aggregating for single computer lets select specific(not needed but for time constraint required)
        .where("year(InvoiceDate) == 2010") \
        .withColumn("WeekNumber", f.weekofyear(f.col("InvoiceDate"))) \ #toadd new column extracting weekno is as simple as using weekofyear function
        .groupBy("Country", "WeekNumber") \ #groupby country and weekno
        .agg(NumInvoices, TotalQuantity, InvoiceValue)

In [4]:
exSummary_df = invoice_df \
        .withColumn("InvoiceDate", f.to_date(f.col("InvoiceDate"), "dd-MM-yyyy H.mm")) \
        .where("year(InvoiceDate) == 2010") \
        .withColumn("WeekNumber", f.weekofyear(f.col("InvoiceDate"))) \
        .groupBy("Country", "WeekNumber") \
        .agg(NumInvoices, TotalQuantity, InvoiceValue)

In [5]:
# save this data file as parquet file and use it in next
exSummary_df.coalesce(1) \
        .write \
        .format("parquet") \
        .mode("overwrite") \
        .save("output")

In [6]:
# sort datframe and show
exSummary_df.sort("Country", "WeekNumber").show()

+---------------+----------+-----------+-------------+------------+
|        Country|WeekNumber|NumInvoices|TotalQuantity|InvoiceValue|
+---------------+----------+-----------+-------------+------------+
|      Australia|        48|          1|          107|      358.25|
|      Australia|        49|          1|          214|       258.9|
|      Australia|        50|          2|          133|      387.95|
|        Austria|        50|          2|            3|      257.04|
|        Bahrain|        51|          1|           54|      205.74|
|        Belgium|        48|          1|          528|       346.1|
|        Belgium|        50|          2|          285|      625.16|
|        Belgium|        51|          2|          942|      838.65|
|Channel Islands|        49|          1|           80|      363.53|
|         Cyprus|        50|          1|          917|     1590.82|
|        Denmark|        49|          1|          454|      1281.5|
|           EIRE|        48|          7|        

In [ ]:
spark.stop()